In [1]:
from google.colab import drive 
drive.mount('gdrive')

Mounted at gdrive


In [173]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import layers, models, backend as K, callbacks
from keras.layers import LSTM
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from sklearn import preprocessing
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from keras import regularizers
from keras.layers import TimeDistributed
import math
from sklearn.metrics import roc_auc_score

In [255]:
stock_market_df = pd.read_csv('/content/gdrive/MyDrive/Stock_market_label_data.csv')
stock_market_df['Match'] = stock_market_df['Match'].replace(['-1'], '0')
stock_market_df.drop(columns=['Company_Name', 'Unnamed: 0', 'Date', 'Name'], inplace=True)
stock_market_df.fillna(0)
stock_market_df = stock_market_df[: 15862]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [256]:
stock_market_df_1 = stock_market_df.drop(columns=['Match'])
X=stock_market_df_1.values.astype(float)
y= stock_market_df['Match'].values.astype(float)

In [213]:
y

array([1., 1., 1., ..., 1., 0., 0.])

In [257]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(X)

In [258]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [259]:
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

12689 3173


In [218]:
look_back = 30
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [243]:
trainX

array([[[0.0534392 , 0.0534392 , 0.0534392 , ..., 0.06393363,
         0.06393363, 0.06393363]],

       [[0.0534392 , 0.0534392 , 0.0534392 , ..., 0.06393363,
         0.06393363, 0.06393363]],

       [[0.0534392 , 0.0534392 , 0.0534392 , ..., 0.06393363,
         0.06393363, 0.06393363]],

       ...,

       [[0.5937898 , 0.59170225, 0.59170225, ..., 0.60602886,
         0.60602886, 0.60602886]],

       [[0.59170225, 0.59170225, 0.59170225, ..., 0.60602886,
         0.60602886, 0.60602886]],

       [[0.59170225, 0.59170225, 0.59170225, ..., 0.60602886,
         0.60602886, 0.60602886]]])

In [242]:
trainY

array([0.06393363, 0.06393363, 0.0663247 , ..., 0.60602886, 0.60602886,
       0.60602886])

In [219]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [220]:
print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

(12658, 1, 30)
(3142, 1, 30)
(12658,)
(3142,)


In [241]:
model = tf.keras.Sequential()
model.add(layers.LSTM(4, input_shape=(1, 30), return_sequences=True))
model.add(TimeDistributed(layers.Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = [tf.keras.metrics.AUC()])
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
12658/12658 - 18s - loss: 0.5591 - auc_10: 0.8899
Epoch 2/100
12658/12658 - 16s - loss: 0.5515 - auc_10: 0.9988
Epoch 3/100
12658/12658 - 16s - loss: 0.5514 - auc_10: 0.9989
Epoch 4/100
12658/12658 - 17s - loss: 0.5514 - auc_10: 0.9994
Epoch 5/100
12658/12658 - 16s - loss: 0.5513 - auc_10: 0.9992
Epoch 6/100
12658/12658 - 16s - loss: 0.5513 - auc_10: 0.9991
Epoch 7/100
12658/12658 - 16s - loss: 0.5513 - auc_10: 0.9990
Epoch 8/100
12658/12658 - 17s - loss: 0.5513 - auc_10: 0.9989
Epoch 9/100
12658/12658 - 16s - loss: 0.5513 - auc_10: 0.9988
Epoch 10/100
12658/12658 - 16s - loss: 0.5513 - auc_10: 0.9990
Epoch 11/100
12658/12658 - 16s - loss: 0.5513 - auc_10: 0.9991
Epoch 12/100
12658/12658 - 16s - loss: 0.5513 - auc_10: 0.9996
Epoch 13/100
12658/12658 - 16s - loss: 0.5513 - auc_10: 0.9996
Epoch 14/100
12658/12658 - 16s - loss: 0.5513 - auc_10: 0.9993
Epoch 15/100
12658/12658 - 16s - loss: 0.5513 - auc_10: 0.9994
Epoch 16/100
12658/12658 - 17s - loss: 0.5513 - auc_10: 0.9992
E

In [223]:
def naive_roc_auc_score(y_true, y_pred):
  num_same_sign = 0
  num_pairs = 0
  for a in range(len(y_true)):
    for b in range(len(y_true)):
      if y_true[a] > y_true[b]:
        num_pairs += 1
        if y_pred[a] > y_pred[b]:
          num_same_sign += 1
        elif y_pred[a] == y_pred[b]:
          num_same_sign += .5
  return num_same_sign / num_pairs

In [260]:
result = model.predict(testX, batch_size=testX.shape[0], verbose=1)

1/1 [==============================] - 0s 347ms/step


In [261]:
y_hat = result.reshape(-1)
y = testY.reshape(-1)

In [262]:
y_hat.shape

(3142,)

In [264]:
y.shape

(3142,)

In [265]:
naive_roc_auc_score(y, y_hat)

0.9934443473808363